In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode
from plotly.offline import iplot as iplot_offline
from plotly.offline import plot as plot_offline
from plotly.plotly import iplot, plot
from plotly.graph_objs import Scatter
# from plotly.tools import FigureFactory as FF
init_notebook_mode() # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook

%matplotlib inline
import matplotlib as plt
plt.style.use('ggplot')
import scripts.daily_data as dd
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import graph_objs

import requests

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
import os, re
import datetime as dt

In [3]:
testset = ['./raw_data/three-month-export_2016-04-04.csv', './raw_data/three-month-export_2016-04-17.csv']

In [4]:
print(testset)

['./raw_data/three-month-export_2016-04-04.csv', './raw_data/three-month-export_2016-04-17.csv']


In [5]:
def get_time_from_filename(filename: str):
    regex = r'[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
    return re.findall(regex, filename)[0]
print(get_time_from_filename(testset[0]))
print(get_time_from_filename(testset[1]))

2016-04-04
2016-04-17


In [6]:
make_df(testset[0]).downloads_df.values[0].num_downloads

NameError: name 'make_df' is not defined

In [7]:
def make_df(filename: str) -> pd.DataFrame:
    df_time = get_time_from_filename(filename)
    df = pd.read_csv(filename, parse_dates=['release_date'])
    data = {
        'title': df.item_title.values,
        'release_date': df.release_date.values,
        'downloads': [[(df_time, num_downloads)]
                      for num_downloads in df.downloads__total.values],
        'downloads_df': [pd.DataFrame({ 'num_downloads': num_downloads}, index=[df_time])
                                      for num_downloads in df.downloads__total.values]
    }
    return pd.DataFrame(data)

In [8]:
def update_df_data(base_df, new_df):
    for index, row in new_df.iterrows():
        if row.title in base_df.title.values:
            if row.downloads not in base_df[base_df.title == row.title].downloads.values:
                base_df[base_df.title == row.title].downloads.values[0].append(row.downloads[0])
                base_df.loc[base_df.title == row.title, 'downloads_df'] = pd.concat(
                    [base_df.loc[base_df.title == row.title, 'downloads_df'].values[0],
                        row.downloads_df])
        else:
            base_df = base_df.append(row)
#         print(pd.concat([base_df.loc[base_df.title == row.title, 'downloads_df'].values[0],
#                         row.downloads_df]))
#     print(type(base_df.loc[base_df.title == row.title, 'downloads_df'].values[0]))
    return base_df

In [9]:
base_df = pd.DataFrame(columns=['title', 'release_date', 'downloads', 'downloads_df'])

In [10]:
path = './raw_data/'
extensions = ('.csv')

if os.path.isdir(path):
    for root, dirs, files in os.walk(os.path.abspath(path)):
        for file in files:
            if file.split('.')[-1] in extensions and 'three' in file:
                filepath = os.path.join(root,file)
                base_df = update_df_data(base_df, make_df(filepath))
base_df = base_df.sort_values(by='release_date').reset_index()

TypeError: cannot concatenate a non-NDFrame object

In [107]:
base_df.downloads_df.values[0]

nan

In [103]:
base_df

,index,title,release_date,downloads,downloads_df
0,79,Episode #1 - Introduction,2015-06-01 00:34:44,"[(2016-08-14, 1106), (2017-09-23, 3009), (2017...",NaN
1,78,TBP02 - Holding Up the Sky,2015-06-07 23:17:25,"[(2016-08-14, 712), (2017-09-23, 2511), (2017-...",NaN
2,77,TBP03 - Bridging the Gender Gap,2015-06-15 01:30:08,"[(2016-08-14, 574), (2017-09-23, 2374), (2017-...",NaN
3,76,TBP04 - Change Jar 2.0,2015-06-20 12:57:42,"[(2016-08-14, 601), (2017-09-23, 2484), (2017-...",NaN
4,75,TBP05 - The World's Strongest,2015-06-28 22:55:15,"[(2016-08-14, 510), (2017-09-23, 2373), (2017-...",NaN
5,74,TBP06 - Aboard the Pequod,2015-07-02 15:44:18,"[(2016-08-14, 437), (2017-09-23, 2294), (2017-...",NaN
6,73,TBP07 - Under the Iron Veil,2015-07-06 13:30:54,"[(2016-08-14, 494), (2017-09-23, 2536), (2017-...",NaN
7,72,TBP08 - Bitcoin For The Bottom Billion,2015-07-09 20:35:37,"[(2016-08-14, 570), (2017-09-23, 678), (2017-1...",NaN
8,71,TBP09 - The Curious Case of Remittance,2015-07-19 18:11:43,"[(2016-08-14, 503), (2017-09-23, 604), (2017-1...",NaN
9,70,TBP10 - Bottoms Up,2015-07-24 13:32:59,"[(2016-08-14, 440), (2017-09-23, 525), (2017-1...",NaN
